<a href="https://colab.research.google.com/github/janlucasdeinhard/pyfiles/blob/master/DB_QualityCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,time,math,itertools,os,sys

import torch
import torch.nn as nn
from torch.nn import functional as F

from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Concatenate,Input
from tensorflow.keras.regularizers import l2

import pandas as pd

import snowflake.connector

import plotly as pl

In [2]:
conn = snowflake.connector.connect(
    user='jan-lucas.deinhard@siemens-healthineers.com',
    account='shsitdl.west-europe.azure',
    authenticator='externalbrowser'
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/5dbf1add-202a-4b8d-815b-bf0fb024e033/saml2?SAMLRequest=nZJbc5swEIX%2FCqM%2BAwLfNbYzTmxPGOfiGtJO%2BybQYmsMEpVEcPLrK7A9kz4kD33TSGf3O9qz05tTWTivoDSXYoYCDyMHRCYZF%2FsZeknW7hg52lDBaCEFzNAbaHQzn2paFhVZ1OYgdvCnBm0c20ho0j7MUK0EkVRzTQQtQROTkXjx%2BEBCDxOqNShjcehSwjS3rIMxFfH9pmm8pudJtfdDjLGPJ75VtZJv6AOi%2BppRKWlkJotrycn%2B6RNE4ON%2Bi7AKS9heCm%2B5OI%2FgK0p6FmlynyRbd%2FscJ8hZXH93J4WuS1AxqFeewcvu4WxAWwfxfRwlywevsXNzoVayAo%2B%2B1wo8LWSTF%2FQImSyr2tjunj35OTC%2FkHtuZxYtZ6g6crb5SVf5kJ76g8Nwc6xGW%2FaMV796I%2Fp9sruNN4vHan14T1U0glWGnB%2FXhMM24UjrGiLR5mrsFQ5DF%2FfcYJIEQxJiEvS9YDD5jZyl9ccFNV3l1X

KeyboardInterrupt: ignored

In [ ]:
info = pd.read_sql("SHOW COLUMNS",con=conn)

In [ ]:
df = info[info['database_name']=='CRMCL_TOPICAREA']

df['table_name'] = df[['schema_name','table_name']].apply(lambda x: x[0]+'.'+x[1],axis=1)
df = df[df['schema_name'].isin([
    'AUXFILES',
    'CNJ',
    'CPQ',
    'GCR',
    'OSC',
    'VARIAN'
])].drop(columns=['data_type','null?','kind','expression','comment','database_name','autoincrement','default','schema_name'])

<ipython-input-20-b893168c8783>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['table_name'] = df[['schema_name','table_name']].apply(lambda x: x[0]+'.'+x[1],axis=1)


In [ ]:
conn.close()

In [ ]:
df

,table_name,column_name
627244,AUXFILES.Account_Address_Geolocations,ACCOUNT_ID
627245,AUXFILES.Account_Address_Geolocations,Account Address
627246,AUXFILES.Account_Address_Geolocations,Account City
627247,AUXFILES.Account_Address_Geolocations,Account.Address Country
627248,AUXFILES.Account_Address_Geolocations,Account.Address accuracy
...,...,...
635303,VARIAN.VarianTerritories,Sub-Region
635304,VARIAN.VarianTerritories,Super-Territory
635305,VARIAN.VarianTerritories,TERRITORY_ID
635306,VARIAN.VarianTerritories,Territory


In [ ]:
df0 = df.groupby(['column_name'],as_index=False).agg({'table_name':'||'.join})

In [ ]:
df0['num_tables_with_key'] = df0['table_name'].apply(lambda x: len(str(x).split('||')))

In [ ]:
df0 = df0.sort_values(by='num_tables_with_key',ascending=False)

In [ ]:
df0[df0['num_tables_with_key']>1]

,column_name,table_name,num_tables_with_key
1,ACCOUNT_ID,AUXFILES.Account_Address_Geolocations||AUXFILE...,54
1784,SECURITY_ID,CNJ.CNJ_MarketShare||CPQ.ACTIVE_QUOTES_HEADER|...,45
1672,Quote Number and Revision,CPQ.ACTIVE_QUOTES_HEADER||CPQ.CS_QUOTEITEM_COS...,22
1002,Group Currency,CPQ.CS_QUOTEITEM_COSTS||CPQ.CS_Quote_Details||...,20
354,Business Line,AUXFILES.Manual Adjustments||CPQ.IN_VITRO_Addi...,18
...,...,...,...
509,Consumable Name,CPQ.IN_VITRO_Consumables_per_Enduser||CPQ.IN_V...,2
241,Asset Age (days),OSC.Assets||VARIAN.VarianAssets,2
238,Assay Name,AUXFILES.IN_VITRO_AssayNameMap_by_PartNumber||...,2
252,Asset Install Year,OSC.Assets||VARIAN.VarianAssets,2


In [ ]:
df1 = df[df['column_name'].isin(df0[df0['column_name'].str.contains('_ID')]['column_name'].unique())]

In [ ]:
rf = pd.DataFrame()

for ccol in df0[df0['column_name'].str.contains('_ID')]['column_name'].unique():
    connected_cols = df1[df1['table_name'].isin(df1[df1['column_name']==ccol]['table_name'].unique())]['column_name'].unique()
    ff = pd.DataFrame(connected_cols).rename(columns={0:'From'})
    ff['To'] = ccol
    rf = pd.concat([rf,ff])

In [ ]:
rf['FromNEW'] = rf.apply(lambda x: sorted([x[0],x[1]])[0],axis=1)
rf['ToNEW'] = rf.apply(lambda x: sorted([x[0],x[1]])[1],axis=1)

In [ ]:
rf = rf.drop(columns=['From','To']).rename(columns={'FromNEW':'From','ToNEW':'To'}).drop_duplicates()

In [ ]:
rf = rf[~rf.apply(lambda x: x[0]==x[1],axis=1)]

In [ ]:
rf

,From,To
1,ACCOUNT_ID,VAR_ACCOUNT_ID
2,ACCOUNT_ID,OPPORTUNITY_ID
3,ACCOUNT_ID,PROJECT_ID
4,ACCOUNT_ID,SECURITY_ID
5,ACCOUNT_ID,OLI_ID
...,...,...
9,EMPLOYEE_ID (Account Primary Sales Rep POC),EMPLOYEE_ID (Account Secondary Sales Rep In-Vivo)
8,EMPLOYEE_ID (Account Primary Sales Rep Ultraso...,EMPLOYEE_ID (Account Secondary Sales Rep In-Vi...
9,EMPLOYEE_ID (Account Primary Sales Rep Ultraso...,EMPLOYEE_ID (Account Secondary Sales Rep In-Vivo)
9,EMPLOYEE_ID (Account Secondary Sales Rep In-Vi...,EMPLOYEE_ID (Account Secondary Sales Rep In-Vivo)


In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
G.add_edges_from(rf[['From','To']].apply(lambda x: (x[0],x[1]),axis=1).tolist())

In [ ]:
G.degree